In [119]:
%pip install pandas
%pip install scikit-learn
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [120]:
folder_path = "teamCsvFiles/"
csvArray = []
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        csvArray.append({"teamName":file_name,"data":pd.read_csv(file_path, encoding='latin1')})

In [121]:
csvArray[0]["data"]

,gameId,teamId,opponentId,homeTeam?,playerInfoId,playerFullName,battingRuns,battingDoubles,battingTriples,battingHomeRuns,...,pitchingAtBats,pitchingStolenBases,pitchingNumberOfPitches,pitchingInningsPitched,pitchingWins,pitchingLosses,pitchingHolds,pitchingBlownSaves,pitchingEarnedRuns,pitchingPitchesThrown
0,717223,109,136,True,645444,Jose Herrera,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,717223,109,136,True,543518,Scott McGough,NaN,NaN,NaN,NaN,...,2.0,0.0,9.0,0.2,0.0,0.0,0.0,0.0,0.0,9.0
2,717223,109,136,True,694297,Brandon Pfaadt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,717223,109,136,True,669459,Kyle Nelson,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,717223,109,136,True,518876,Merrill Kelly,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4252,747954,109,158,False,572233,Christian Walker,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4253,747954,109,158,False,462101,Elvis Andrus,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4254,747954,109,158,False,699228,Jakey Josepha,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4255,747954,109,158,False,656896,Emmanuel Rivera,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
def formatPlayerStats(playerdf):
    game_scores = playerdf.groupby('gameId').agg(
    teamScore=('battingRuns', 'sum'),
    opponentScore=('pitchingRuns', 'sum')
)
    # Assign 'Win' or 'Loss' based on the comparison of scores
    game_scores['result'] = game_scores.apply(lambda row: 'Win' if row['teamScore'] > row['opponentScore'] else 'Loss', axis=1)
    # Merge the result back to the original dataframe based on gameId
    playerdf = playerdf.merge(game_scores['result'], left_on='gameId', right_index=True)
    playerdf["battingAvg"] = (playerdf["battingHits"]) / (playerdf["battingAtBats"])
    playerdf["onBasePercentage"] = (playerdf["battingHits"] + playerdf["battingBaseOnBalls"]) / (playerdf["battingAtBats"] + playerdf["battingBaseOnBalls"])
    playerdf.fillna(0, inplace=True) 
    return playerdf

In [123]:
def formatTeamStats(playerdf):
    aggdf = playerdf.groupby(['playerFullName']).sum().reset_index()
    # Drop columns
    aggdf = aggdf.drop(['gameId','opponentId','homeTeam?','playerInfoId'], axis=1)
    teamId = playerdf.iloc[0]["teamId"]  
    aggdf["teamId"] = teamId
    aggdf["battingAvg"] = (aggdf["battingHits"]) / (aggdf["battingAtBats"])
    aggdf["onBasePercentage"] = (aggdf["battingHits"] + aggdf["battingBaseOnBalls"]) / (aggdf["battingAtBats"] + aggdf["battingBaseOnBalls"])
    aggdf.fillna(0, inplace=True)
    return aggdf

In [124]:
teamStatsArray = []
for i in range(len(csvArray)):
    teamStatsArray.append({"teamName":csvArray[i]["teamName"],"data":formatTeamStats(csvArray[i]["data"])})

In [125]:
for i in range(len(csvArray)):
    csvArray[i]["data"] = formatPlayerStats(csvArray[i]["data"])

In [126]:
def addSabermetrics(df):
    df["battingSingles"] = (df["battingHits"] - df["battingDoubles"] - df["battingTriples"] - df["battingHomeRuns"])
    df["totalBases"] = (df["battingSingles"] + 2*df["battingDoubles"] + 3*df["battingTriples"] + 4*df["battingHomeRuns"])
    df["battingSlg"] = (df["totalBases"]/df["battingAtBats"])
    df["OBP+Slg"] = (df["onBasePercentage"] + df["battingSlg"])
    df["pitchingSingles"] = (df["pitchingHits"] - df["pitchingDoubles"] - df["pitchingTriples"] - df["pitchingHomeRuns"])
    df["pitchingRunsAllowedPerInning"] = (df["pitchingEarnedRuns"]/df["pitchingInningsPitched"])
    df["pitchingWinPercentage"] = (df["pitchingWins"] / (df["pitchingWins"] + df["pitchingLosses"]))
    df["pitchingWalks+Hits"] = (df["pitchingHits"] + df["pitchingBaseOnBalls"])
    df["pitchingWalks+HitsPercentage"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingAtBats"])
    df["pitchingWalks+HitsPerInning"] = ((df["pitchingHits"] + df["pitchingBaseOnBalls"]) / df["pitchingInningsPitched"])
    df.fillna(0, inplace=True)
    

In [127]:
for i in range(len(csvArray)):
    addSabermetrics(csvArray[i]["data"])
    addSabermetrics(teamStatsArray[i]["data"])

In [128]:
for i in range(len(csvArray)):
    csvArray[i]["data"]["homeTeam?"] = csvArray[i]["data"]["homeTeam?"].map({
    True: 1,
    False: 0
})
    
    csvArray[i]["data"]["result"] = csvArray[i]["data"]["result"].map({
    "Win": 1,
    "Loss": 0
})

In [135]:
for i in range(len(csvArray)):
    csvArray[i]["data"].replace([np.inf, -np.inf], 0, inplace=True)
    teamStatsArray[i]["data"].replace([np.inf, -np.inf], 0, inplace=True)

In [130]:
features = list(csvArray[0]["data"].columns[1:5]) 
features.extend(csvArray[0]["data"].columns[6:])

In [138]:
features = list(teamStatsArray[0]["data"].columns[1:])

Index(['teamId', 'battingRuns', 'battingDoubles', 'battingTriples',
       'battingHomeRuns', 'battingStrikeOuts', 'battingBaseOnBalls',
       'battingHits', 'battingAtBats', 'battingStolenBases', 'battingRbi',
       'battingLeftOnBase', 'pitchingRuns', 'pitchingDoubles',
       'pitchingTriples', 'pitchingHomeRuns', 'pitchingStrikeOuts',
       'pitchingBaseOnBalls', 'pitchingHits', 'pitchingAtBats',
       'pitchingStolenBases', 'pitchingNumberOfPitches',
       'pitchingInningsPitched', 'pitchingWins', 'pitchingLosses',
       'pitchingHolds', 'pitchingBlownSaves', 'pitchingEarnedRuns',
       'pitchingPitchesThrown', 'battingAvg', 'onBasePercentage',
       'battingSingles', 'totalBases', 'battingSlg', 'OBP+Slg',
       'pitchingSingles', 'pitchingRunsAllowedPerInning',
       'pitchingWinPercentage', 'pitchingWalks+Hits',
       'pitchingWalks+HitsPercentage', 'pitchingWalks+HitsPerInning'],
      dtype='object')

In [131]:
X_train_dict = teamStatsArray[0]["data"][features].to_dict(orient="records")

In [132]:
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer(sparse=False)
vec.fit(X_train_dict)
X_train = vec.transform(X_train_dict)
x_new_dict = {
    "teamId": 136,
    "homeTeam?": 0,
    "opponentId": 109
}
X_new = vec.transform([x_new_dict])

In [133]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_new_sc = scaler.transform(X_new)

X_train_sc

array([[ 0.68367318,  2.26806677,  1.07636021, ..., -0.8153782 ,
         0.        ,  0.76977669],
       [-0.35731571, -0.54234226, -0.35036336, ..., -0.8153782 ,
         0.        , -0.31710204],
       [-0.35731571, -0.54234226, -0.35036336, ..., -0.8153782 ,
         0.        , -0.31710204],
       ...,
       [-0.35731571, -0.54234226, -0.35036336, ..., -0.8153782 ,
         0.        , -0.31710204],
       [-0.35731571, -0.54234226, -0.35036336, ..., -0.8153782 ,
         0.        , -0.31710204],
       [-0.35731571, -0.54234226, -0.35036336, ..., -0.8153782 ,
         0.        , -0.31710204]])

In [134]:
from sklearn.neighbors import KNeighborsRegressor
y_train = csvArray[0]["data"]["result"]
model = KNeighborsRegressor(n_neighbors=30)
model.fit(X_train_sc, y_train)
model.predict(X_new_sc)

array([0.3])

In [141]:
%pip install --upgrade pip
%pip install pybaseball
import os
import pandas as pd
import numpy as np



  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.1 MB 656.4 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 1.6 MB/s eta 0:00:02
   ------- -------------------------------- 0.4/2.1 MB 2.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.1 MB 4.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
Note: you may need to restart the k

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python312\\lib\\site-packages\\pip-23.2.1.dist-info\\AUTHORS.txt'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [192]:
START = 2005
END = 2025

In [200]:
from pybaseball import batting_stats
from pybaseball import pitching_stats
from pybaseball import fielding_stats
batting = batting_stats(START,END,qual=200)
pitching = pitching_stats(START,END,qual=50)
fielding = fielding_stats(START,END)

In [201]:
batting.to_csv("batting.csv")
pitching.to_csv("batting.csv")
fielding.to_csv("batting.csv")
csvs = [batting,pitching,fielding]

In [202]:
for i in csvs:
    i = i.groupby("IDfg",group_keys=False).filter(lambda x: x.shape[0] > 1)

In [ ]:
def next_season(player):
    player = player.sort_values("Season")
    player["Next_WAR"] = player["WAR"].shift(-1)
    return player